In [1]:
#这些是你没有import的包
import gc
import pyarrow as pa
from transformers import AutoTokenizer, DataCollatorWithPadding, DataCollatorForLanguageModeling
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import load_metric
import datasets
import transformers
#下面这3段与你一样

In [2]:
from sentence_transformers import SentenceTransformer, LoggingHandler, losses, models, util, InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import InputExample
import torch
from torch import nn
from torch.utils.data import DataLoader

import math
import random
from tqdm import tqdm
from scipy.stats import pearsonr

import numpy as np
import pandas as pd
import os

In [3]:
for dirname, _, filenames in os.walk('./kaggle/input/us-patent-phrase-to-phrase-matching'):
    for filename in filenames:
        if filename == 'train.csv':
            raw_data = pd.read_csv(os.path.join(dirname, filename))
        elif filename == 'test.csv':
            test_data = pd.read_csv(os.path.join(dirname, filename))
        elif filename == 'sample_submission.csv':
            sample_submission = pd.read_csv(os.path.join(dirname, filename))

In [4]:
def split_train_eval(data, train_per=0.8, train_num=None):
    if train_num is None:
        train_num = int(len(data) * train_per)
        
    raw_data_index = list(range(len(data)))
    random.shuffle(raw_data_index)
    train_data = raw_data.loc[raw_data_index[:train_num]]
    train_data = train_data.sort_index(ascending=True)
    eval_data = raw_data.loc[raw_data_index[train_num:]]
    eval_data = eval_data.sort_index(ascending=True)
    return train_data, eval_data

train_data, eval_data = split_train_eval(raw_data, train_num=len(raw_data) - 37 * 4)

In [5]:
def FrameToSet(data):
    output_data = pd.DataFrame(columns=['sentence1', 'sentence2', 'context1', 'context2', 'label', 'idx'])
    output_data['sentence1'] = list(data['anchor'])
    output_data['sentence2'] = list(data['target'])
    output_data['label'] = list(data['score'])
    output_data['idx'] = list(np.arange(0,len(output_data['label'])))
    contexts = []
    for x in list(data['context']):
        contexts.append([ord(x[0])-65,10*int(x[1])+int(x[2])])
    output_data['context1'] = list(np.array(contexts)[:,0])
    output_data['context2'] = list(np.array(contexts)[:,1])
    return output_data

In [6]:
def FrameToSet_test(data):
    output_data = pd.DataFrame(columns=['sentence1', 'sentence2', 'context1', 'context2', 'label', 'idx'])
    output_data['sentence1'] = list(data['anchor'])
    output_data['sentence2'] = list(data['target'])
    output_data['label'] = list([0.0]*len(output_data['sentence2']))
    output_data['idx'] = list(np.arange(0,len(output_data['label'])))
    contexts = []
    for x in list(data['context']):
        contexts.append([ord(x[0])-65,10*int(x[1])+int(x[2])])
    output_data['context1'] = list(np.array(contexts)[:,0])
    output_data['context2'] = list(np.array(contexts)[:,1])
    return output_data

In [7]:
the_train, thetest_ = split_train_eval(raw_data)
thetrain_, theeval_ = split_train_eval(the_train)
#如果提交代码，运行下面这段(1/3)
'''
thetrain_, theeval_ = split_train_eval(raw_data, train_num=len(raw_data) - 37 * 4)
thetest_ = test_data
'''
###设置###
model_checkpoint = "microsoft/deberta-v3-small"
#有条件用下面这行
#需要下载
#model_checkpoint = "microsoft/deberta-v3-large"
batch_size = 8#有条件大一些
num_epoch = 3.0#就是float型，默认为3.0

In [62]:
#DeBERTa模型       这是不按ABCD分类的
thetrain = datasets.arrow_dataset.Dataset(pa.Table.from_pandas(FrameToSet(thetrain_)))
thetest = datasets.arrow_dataset.Dataset(pa.Table.from_pandas(FrameToSet(thetest_)))
theeval = datasets.arrow_dataset.Dataset(pa.Table.from_pandas(FrameToSet(theeval_)))
thedict = {"train":thetrain,"validation":theeval,"test":thetest}
dataset = datasets.dataset_dict.DatasetDict(thedict)
#如果提交代码，运行下面这段(3/3)
'''
thetrain = datasets.arrow_dataset.Dataset(pa.Table.from_pandas(FrameToSet(thetrain_)))
thetest = datasets.arrow_dataset.Dataset_test(pa.Table.from_pandas(FrameToSet(thetest_)))#这行不一样
theeval = datasets.arrow_dataset.Dataset(pa.Table.from_pandas(FrameToSet(theeval_)))
thedict = {"train":thetrain,"validation":theeval,"test":thetest}
dataset = datasets.dataset_dict.DatasetDict(thedict)
'''

gc.collect()
torch.cuda.empty_cache()

GLUE_TASKS = ["stsb"]
for task in GLUE_TASKS:
    
    #List of glue keys
    task_to_keys = {
        "stsb": ("sentence1", "sentence2"),
    }
    
    #Select task
    #task = "rte"  #cola, mrpc
    #batch_size = 8 #10 normally, 8 for qnli
    
    # Load dataset based on task variable
    actual_task = task
    #dataset = load_dataset("glue", actual_task)
    metric = load_metric('glue', actual_task)#这行需要下载东西########################################################################
    
    #Collect sentence keys and labels
    sentence1_key, sentence2_key = task_to_keys[task]
    
    # Number of logits to output
    num_labels = 1
    
    ###  Tokenizing Section  ####
    
    #Load model
    #model_checkpoint = "microsoft/deberta-v3-small"
    
    # Create tokenizer for respective model
    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True, truncation=True, model_max_length=512)
    
    def tokenizer_func(examples):
        if sentence2_key is None:
            return tokenizer(examples[sentence1_key], truncation=True,)
        return tokenizer(examples[sentence1_key], examples[sentence2_key], truncation=True,)
    
    # tokenize sentence(s)
    encoded_dataset = dataset.map(tokenizer_func, batched=True)
    
    '''
    #model_checkpoint = "deberta-v3-small_baseline_cola/"
    model_checkpoint = "deberta-v3-small_baseline_"+actual_task+"/"
    '''
    ###  Model Section  ####
    
    # Create model and attach ForSequenceClassification head
    model_deberta = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)
    ########################模型定义在上面这行
    # Type of metric for given task
    metric_name = "pearson"
    
    args = TrainingArguments(
        f"{model_checkpoint}-finetuned-Testing-{task}",
        evaluation_strategy = "epoch",
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        weight_decay=0.01,
        metric_for_best_model=metric_name,
        eval_accumulation_steps=5,
        num_train_epochs=num_epoch
    )
    
    def compute_metrics(eval_pred):
        predictions, labels = eval_pred
        predictions = predictions[:]#, 0]
        return metric.compute(predictions=predictions, references=labels)
    
    validation_key = "validation_mismatched" if task == "mnli-mm" else "validation_matched" if task == "mnli" else "validation"
    trainer = Trainer(
        model_deberta,
        args,
        train_dataset=encoded_dataset["train"],
        eval_dataset=encoded_dataset[validation_key],
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )
    trainer.train()
    trainer.evaluate()


    
    ### Collect Predictions  ###
    
    prediction_one = trainer.predict(encoded_dataset["test"])
    

    
    ## Clear the Cache
    del trainer
    del args
    del model_deberta
    del encoded_dataset
    del dataset
    del thedict
    del theeval
    del thetest
    del thetrain
    gc.collect()
    torch.cuda.empty_cache()

loading configuration file https://huggingface.co/microsoft/deberta-v3-small/resolve/main/config.json from cache at C:\Users\Mottled-panpipe/.cache\huggingface\transformers\8e0c12a7672d1d36f647c86e5fc3a911f189d8704e2bc94dde4a1ffe38f648fa.9df96bac06c2c492bc77ad040068f903c93beec14607428f25bf9081644ad0da
Model config DebertaV2Config {
  "_name_or_path": "microsoft/deberta-v3-small",
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-07,
  "max_position_embeddings": 512,
  "max_relative_positions": -1,
  "model_type": "deberta-v2",
  "norm_rel_ebd": "layer_norm",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 0,
  "pooler_dropout": 0,
  "pooler_hidden_act": "gelu",
  "pooler_hidden_size": 768,
  "pos_att_type": [
    "p2c",
    "c2p"
  ],
  "position_biased_input": false,
  "position_buckets": 256,
  "relative_attention":

  0%|          | 0/24 [00:00<?, ?ba/s]

  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/8 [00:00<?, ?ba/s]

loading configuration file https://huggingface.co/microsoft/deberta-v3-small/resolve/main/config.json from cache at C:\Users\Mottled-panpipe/.cache\huggingface\transformers\8e0c12a7672d1d36f647c86e5fc3a911f189d8704e2bc94dde4a1ffe38f648fa.9df96bac06c2c492bc77ad040068f903c93beec14607428f25bf9081644ad0da
Model config DebertaV2Config {
  "_name_or_path": "microsoft/deberta-v3-small",
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-07,
  "max_position_embeddings": 512,
  "max_relative_positions": -1,
  "model_type": "deberta-v2",
  "norm_rel_ebd": "layer_norm",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 0,
  "pooler_dropout": 0,
  "pooler_hidden_act": "gelu",
  "pooler_hidden_size": 768,
  "pos_att_type": [
    "p2c",
    "c2p"
  ],
  "pos

Epoch,Training Loss,Validation Loss,Pearson,Spearmanr
1,0.031600,0.027126,0.803076,0.784219
2,0.019400,0.024693,0.826902,0.809539
3,0.013500,0.021472,0.838057,0.819464


Saving model checkpoint to microsoft/deberta-v3-small-finetuned-Testing-stsb\checkpoint-500
Configuration saved in microsoft/deberta-v3-small-finetuned-Testing-stsb\checkpoint-500\config.json
Model weights saved in microsoft/deberta-v3-small-finetuned-Testing-stsb\checkpoint-500\pytorch_model.bin
tokenizer config file saved in microsoft/deberta-v3-small-finetuned-Testing-stsb\checkpoint-500\tokenizer_config.json
Special tokens file saved in microsoft/deberta-v3-small-finetuned-Testing-stsb\checkpoint-500\special_tokens_map.json
Saving model checkpoint to microsoft/deberta-v3-small-finetuned-Testing-stsb\checkpoint-1000
Configuration saved in microsoft/deberta-v3-small-finetuned-Testing-stsb\checkpoint-1000\config.json
Model weights saved in microsoft/deberta-v3-small-finetuned-Testing-stsb\checkpoint-1000\pytorch_model.bin
tokenizer config file saved in microsoft/deberta-v3-small-finetuned-Testing-stsb\checkpoint-1000\tokenizer_config.json
Special tokens file saved in microsoft/deberta

Special tokens file saved in microsoft/deberta-v3-small-finetuned-Testing-stsb\checkpoint-7000\special_tokens_map.json
Saving model checkpoint to microsoft/deberta-v3-small-finetuned-Testing-stsb\checkpoint-7500
Configuration saved in microsoft/deberta-v3-small-finetuned-Testing-stsb\checkpoint-7500\config.json
Model weights saved in microsoft/deberta-v3-small-finetuned-Testing-stsb\checkpoint-7500\pytorch_model.bin
tokenizer config file saved in microsoft/deberta-v3-small-finetuned-Testing-stsb\checkpoint-7500\tokenizer_config.json
Special tokens file saved in microsoft/deberta-v3-small-finetuned-Testing-stsb\checkpoint-7500\special_tokens_map.json
Saving model checkpoint to microsoft/deberta-v3-small-finetuned-Testing-stsb\checkpoint-8000
Configuration saved in microsoft/deberta-v3-small-finetuned-Testing-stsb\checkpoint-8000\config.json
Model weights saved in microsoft/deberta-v3-small-finetuned-Testing-stsb\checkpoint-8000\pytorch_model.bin
tokenizer config file saved in microsoft/

The following columns in the test set don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: idx, sentence1, sentence2, context1, context2. If idx, sentence1, sentence2, context1, context2 are not expected by `DebertaV2ForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 7295
  Batch size = 8


In [63]:
#prediction_one.predictions是最终结果
pearsonr(prediction_one.predictions,np.array(ftest["label"]))[0]

0.880908879296934

In [79]:
p = []
for idx in feval.loc[feval["context1"]==7]['idx']:
    p.append(prediction_one.predictions[idx])

In [81]:
np.array(p)

array([0.3982123 , 0.18381532, 0.656159  , ..., 0.5411603 , 0.29826367,
       0.4922697 ], dtype=float32)

In [85]:
ftest.loc[ftest["context1"]>7]['idx']

Series([], Name: idx, dtype: int32)

In [8]:
#DeBERTa模型       按ABCD分类
ftrain = FrameToSet(thetrain_)
ftest = FrameToSet(thetest_)
feval = FrameToSet(theeval_)
#如果提交代码，运行下面这段(2/3)
'''
ftrain = FrameToSet(thetrain_)
ftest = FrameToSet_test(thetest_)#这行不一样
feval = FrameToSet(theeval_)
'''

pre = []

metric = load_metric('glue', "stsb")#这行需要下载东西##########################################################################
#Load model
#model_checkpoint = "microsoft/deberta-v3-small"
# Create tokenizer for respective model
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True, truncation=True, model_max_length=512)
    
def tokenizer_func(examples):
    if sentence2_key is None:
        return tokenizer(examples[sentence1_key], truncation=True,)
    return tokenizer(examples[sentence1_key], examples[sentence2_key], truncation=True,)


gc.collect()
torch.cuda.empty_cache()
for i in range(9):
    if i == 8:#bigger context not in A~H
        all_id = ftest.loc[ftest["context1"]>7]['idx']
        if len(all_id) > 0:
            p = []
            for idx in all_id:
                p.append(prediction_one.predictions[idx])
            pre.append(np.vstack((np.array(p), np.array(ftest.loc[ftest["context1"]>7]['idx']))))
        break
    thetrain = datasets.arrow_dataset.Dataset(pa.Table.from_pandas(ftrain.loc[ftrain["context1"]==i]))
    thetest = datasets.arrow_dataset.Dataset(pa.Table.from_pandas(ftest.loc[ftest["context1"]==i]))
    theeval = datasets.arrow_dataset.Dataset(pa.Table.from_pandas(feval.loc[feval["context1"]==i]))
    thedict = {"train":thetrain,"validation":theeval,"test":thetest}
    dataset = datasets.dataset_dict.DatasetDict(thedict)
    task = "stsb"
    #List of glue keys
    task_to_keys = {
        "stsb": ("sentence1", "sentence2"),
    }
    #Select task
    #task = "rte"  #cola, mrpc
    #batch_size = 8 #10 normally, 8 for qnli
    
    # Load dataset based on task variable
    #dataset = load_dataset("glue", actual_task)
    
    #Collect sentence keys and labels
    sentence1_key, sentence2_key = task_to_keys[task]
    
    # Number of logits to output
    num_labels = 1
    
    ###  Tokenizing Section  ####
    # tokenize sentence(s)
    encoded_dataset = dataset.map(tokenizer_func, batched=True)
    
    '''
    #model_checkpoint = "deberta-v3-small_baseline_cola/"
    model_checkpoint = "deberta-v3-small_baseline_"+actual_task+"/"
    '''
    ###  Model Section  ####
    
    # Create model and attach ForSequenceClassification head
    model_deberta = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)
    ########################模型定义在上面这行
    
    # Type of metric for given task
    metric_name = "pearson"
    
    args = TrainingArguments(
        f"{model_checkpoint}-finetuned-Testing-{task}",
        evaluation_strategy = "epoch",
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        weight_decay=0.01,
        metric_for_best_model=metric_name,
        eval_accumulation_steps=5,
        num_train_epochs=num_epoch
    )
    
    def compute_metrics(eval_pred):
        predictions, labels = eval_pred
        predictions = predictions[:]#, 0]
        return metric.compute(predictions=predictions, references=labels)
    
    validation_key = "validation"
    trainer = Trainer(
        model_deberta,
        args,
        train_dataset=encoded_dataset["train"],
        eval_dataset=encoded_dataset[validation_key],
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )
    trainer.train()#训练过程


    
    ### Collect Predictions  ###
    
    predicts = trainer.predict(encoded_dataset["test"])
    pre.append(np.vstack((predicts.predictions, np.array(ftest.loc[ftest["context1"]==i]['idx']))))

    
    ## 清理gpu，不然容易炸
    del predicts
    del trainer
    del args
    del model_deberta
    del encoded_dataset
    del dataset
    del thedict
    del theeval
    del thetest
    del thetrain
    gc.collect()
    torch.cuda.empty_cache()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
D:\anaconda3\lib\site-packages\transformers\convert_slow_tokenizer.py:434: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

Some weights of the model checkpoint at microsoft/deberta-v3-small were not used when initializing DebertaV2ForSequenceClassification: ['mask_predictions.dense.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.dense.weight', 'mask_predictions.dense.weight', 'mask_predictions.LayerNorm.bias', 'mask_predictions.classifier.weight', 'mask_predictions.LayerNorm.weight', 'mask_predictions.classifier.bias', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.LayerNorm.bias']
- This IS expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from 

...

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

loading configuration file https://huggingface.co/microsoft/deberta-v3-small/resolve/main/config.json from cache at C:\Users\Mottled-panpipe/.cache\huggingface\transformers\8e0c12a7672d1d36f647c86e5fc3a911f189d8704e2bc94dde4a1ffe38f648fa.9df96bac06c2c492bc77ad040068f903c93beec14607428f25bf9081644ad0da
Model config DebertaV2Config {
  "_name_or_path": "microsoft/deberta-v3-small",
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-07,
  "max_position_embeddings": 512,
  "max_relative_positions": -1,
  "model_type": "deberta-v2",
  "norm_rel_ebd": "layer_norm",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 0,
  "pooler_dropout": 0,
  "pooler_hidden_act": "gelu",
  "pooler_hidden_size": 768,
  "pos_att_type": [
    "p2c",
    "c2p"
  ],
  "pos

Epoch,Training Loss,Validation Loss,Pearson,Spearmanr
1,No log,0.042476,0.689251,0.698220
2,0.054900,0.036553,0.767487,0.760136
3,0.030000,0.026343,0.783792,0.774116


The following columns in the evaluation set don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: idx, sentence1, sentence2, __index_level_0__, context1, context2. If idx, sentence1, sentence2, __index_level_0__, context1, context2 are not expected by `DebertaV2ForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1004
  Batch size = 8
Saving model checkpoint to microsoft/deberta-v3-small-finetuned-Testing-stsb\checkpoint-500
Configuration saved in microsoft/deberta-v3-small-finetuned-Testing-stsb\checkpoint-500\config.json
Model weights saved in microsoft/deberta-v3-small-finetuned-Testing-stsb\checkpoint-500\pytorch_model.bin
tokenizer config file saved in microsoft/deberta-v3-small-finetuned-Testing-stsb\checkpoint-500\tokenizer_config.json
Special tokens file saved in microsoft/deberta-v3-small-finetuned-Testing-stsb\checkpoint-500\special_tokens_map.json
The foll

In [59]:
#将各模型结果按原顺序排好
spre = pre[0]
for i in range(1,len(pre)):
    spre = np.concatenate((spre, pre[i]), axis=1)
spre = spre[:,spre[1].argsort()]

In [61]:
#sper[0]是最终的预测结果
pearsonr(spre[0],np.array(ftest["label"]))[0]

0.8705169600300973

In [65]:
a = spre[0]
b = prediction_one.predictions

In [74]:
ap = 0.4
print(pearsonr(ap*a+(1-ap)*b,np.array(ftest["label"]))[0])

0.8919751458041703


In [13]:
#模型保存过程，很粗糙，你可以换着弄
output_model_file = "my_own_model_file.bin"
output_config_file = "my_own_config_file.bin"
output_vocab_file = "my_own_vocab_file.bin"

model_to_save =  model_deberta.module if hasattr( model_deberta, 'module') else  model_deberta

torch.save(model_to_save.state_dict(), output_model_file)
model_to_save.config.to_json_file(output_config_file)
tokenizer.save_vocabulary(output_vocab_file)

Vocabulary path (my_own_vocab_file.bin) should be a directory


In [ ]:
'''
#Bert模型示例
config = BertConfig.from_json_file(output_config_file)
model = BertForQuestionAnswering(config)
state_dict = torch.load(output_model_file)
model.load_state_dict(state_dict)
tokenizer = BertTokenizer(output_vocab_file, do_lower_case=args.do_lower_case)

#GPT模型示例
config = OpenAIGPTConfig.from_json_file(output_config_file)
model = OpenAIGPTDoubleHeadsModel(config)
state_dict = torch.load(output_model_file)
model.load_state_dict(state_dict)
tokenizer = OpenAIGPTTokenizer(output_vocab_file)
'''